## Import

In [1]:
import trimesh as tm
import topogenesis as tg

import os
import math



## Loading context

In [2]:
path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height TODO: will be USER INPUT)

## Presets

In [3]:
vox_size = 10
fh = 3.6 # floor height
FSI_req = 4 # goal for the FSI TODO: USER INPUT

max_extents_height = abs(buildingplot.bounds[0][2]) + abs(buildingplot.bounds[1][2])
max_extents_width = abs(buildingplot.bounds[0][0]) + abs(buildingplot.bounds[1][0])
max_extents_length = abs(buildingplot.bounds[0][1]) + abs(buildingplot.bounds[1][1])

base = buildingplot.apply_transform(tm.transformations.projection_matrix((0,0,0), (0,0,-1))) # project mesh on z plane to get footprint <-- TODO: easier way?
plot_area = base.area/2 # area calculates both sides of the flattened mesh, we need only 1

# Annual solar energy output of PV is calculated as follows:
# E(kwh) = A*r*H*PR where A = total solar panel area, r = solar panel yield/efficiency, H = Annual average solar radiation, PR = performance ratio
# from this site TODO find better source: https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems 

r = 0.15 # yield/efficiency estimate
H = 1050 # to be updated, currently wikipedia estimate. kWh.y
PR = 0.75 # default value, estimate performance ratio influenced by shading, temperature, dust etc.
PV_req = 20 # kWh/m2/year TODO: change to actual expected value from included colors + find source on this value

## Objective functions

In [4]:
def FSI_lowres(floors, width, length, plot = plot_area, goal = FSI_req):
    FSI = (floors * width * length) / plot
    FSI_normalized = 1/(1+(FSI - goal)**2)
    if FSI >= goal:
        FSI_normalized = 1 # if FSI is more than needed, normalized value is 1
    return FSI, FSI_normalized # gives a rough estimate of the FSI, and normalizes this value on 0-1 depending on how close to the goal it is. 1 if it exceeds the goal

def PV_lowres(floors, width, length, rotation, goal = PV_req):
    A = width * length
    E = r * H * PR * A
    return PV, PV_normalized

## Matrices of alternatives